In [7]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
# 数据预处理
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = [
            os.path.join(root_dir, f) 
            for f in os.listdir(root_dir) 
            if os.path.isfile(os.path.join(root_dir, f)) 
            and f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.transform = transform
        
        # 计算标签范围用于归一化
        self.labels = []
        for f in self.image_files:
            try:
                label = int(os.path.basename(f).split('.')[0][:2])
                self.labels.append(label)
            except:
                pass
        self.max_label = max(self.labels) if self.labels else 1
        self.min_label = min(self.labels) if self.labels else 0
    def __len__(self):
        return len(self.image_files)
    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        try:
            image = Image.open(image_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            
            # 从文件名提取标签并归一化
            filename = os.path.basename(image_path)
            label = int(filename.split('.')[0][:2])
            label_normalized = (label - self.min_label) / (self.max_label - self.min_label + 1e-7)
            
            return image, torch.tensor(label_normalized, dtype=torch.float32)
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")
            blank_image = torch.zeros(3, 224, 224)
            return blank_image, torch.tensor(0.5, dtype=torch.float32)
# 初始化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = r'data'  # 替换为你的数据路径
model_save_path = r'model/resnet50_regression.pth'  # 模型保存路径
# 创建保存目录
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
# 数据加载
train_dataset = CustomImageDataset(root_dir=data_dir, transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# 模型配置
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)
model = model.to(device)
# 训练设置
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
# 训练循环
num_epochs = 10
best_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    scheduler.step(avg_loss)
    
    # 保存最佳模型
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'label_range': (train_dataset.min_label, train_dataset.max_label)
        }, model_save_path)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Best: {best_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.2e}')
print(f"Training complete. Best model saved to {model_save_path}")

Epoch 1/10, Loss: 0.1396, Best: 0.1396, LR: 1.00e-03
Epoch 2/10, Loss: 0.0430, Best: 0.0430, LR: 1.00e-03
Epoch 3/10, Loss: 0.0334, Best: 0.0334, LR: 1.00e-03
Epoch 4/10, Loss: 0.0606, Best: 0.0334, LR: 1.00e-03
Epoch 5/10, Loss: 0.0415, Best: 0.0334, LR: 1.00e-03
Epoch 6/10, Loss: 0.0146, Best: 0.0146, LR: 1.00e-03
Epoch 7/10, Loss: 0.0156, Best: 0.0146, LR: 1.00e-03
Epoch 8/10, Loss: 0.0071, Best: 0.0071, LR: 1.00e-03
Epoch 9/10, Loss: 0.0054, Best: 0.0054, LR: 1.00e-03
Epoch 10/10, Loss: 0.0038, Best: 0.0038, LR: 1.00e-03
Training complete. Best model saved to model/resnet50_regression.pth
